## Task

In this compulsory task you will clean the country column and parse the date column in the **store_income_data_task.csv** file.

In [41]:
# Import pandas library
import pandas as pd
# Load up store_income_data.csv
df = pd.read_csv("store_income_data_task.csv")
# Show the first observations
df.head()

,id,store_name,store_email,department,income,date_measured,country
0,1,"Cullen/Frost Bankers, Inc.",NaN,Clothing,$54438554.24,4-2-2006,United States/
1,2,Nordson Corporation,NaN,Tools,$41744177.01,4-1-2006,Britain
2,3,"Stag Industrial, Inc.",NaN,Beauty,$36152340.34,12-9-2003,United States
3,4,FIRST REPUBLIC BANK,ecanadine3@fc2.com,Automotive,$8928350.04,8-5-2006,Britain/
4,5,Mercantile Bank Corporation,NaN,Baby,$33552742.32,21-1-1973,United Kingdom


1. Take a look at all the unique values in the "country" column. Then, convert the column to lowercase and remove any trailing white spaces.

In [42]:
# Get all the unique values for the column "country"
unique_countries = df['country'].unique()
# Print them out and count them
print(f"There are {len(unique_countries)} in the column")
unique_countries

There are 77 in the column


array(['United States/', 'Britain', ' United States', 'Britain/',
       ' United Kingdom', 'U.K.', 'SA ', 'U.K/', 'America',
       'United Kingdom', nan, 'united states', ' S.A.', 'England ', 'UK',
       'S.A./', 'ENGLAND', 'BRITAIN', 'U.K', 'U.K ', 'America/', 'SA.',
       'S.A. ', 'u.k', 'uk', ' ', 'UK.', 'England/', 'england',
       ' Britain', 'united states of america', 'UK/', 'SA/', 'SA',
       'England.', 'UNITED KINGDOM', 'America.', 'S.A..', 's.a.', ' U.K',
       ' United States of America', 'Britain ', 'England', ' SA',
       'United States of America.', 'United States of America/',
       'United States.', 's. africasouth africa', ' England',
       'United Kingdom ', 'United States of America ', ' UK',
       'united kingdom', 'AMERICA', 'America ',
       'UNITED STATES OF AMERICA', ' S. AfricaSouth Africa', 'america',
       'S. AFRICASOUTH AFRICA', 'Britain.', '/', 'United Kingdom.',
       'United States', ' America', 'UNITED STATES', 'sa',
       'United States

In [43]:
# Convert to lower case
df['country'] = df['country'].str.lower()

# Remove trailing white spaces
df['country'] = df['country'].str.strip()

# Replace all null/NaN value country names with 'Other'
df.country.fillna('Other', inplace=True)

2. Note that there should only be three separate countries. Eliminate all variations, so that 'South Africa', 'United Kingdom' and 'United States' are the only three countries.

In [44]:
# Import fuzzywuzzy library to remove variations
import fuzzywuzzy
from fuzzywuzzy import process
import chardet

#define a general purpose function to deal with variations
def replace_matches_in_column(df, column, string_to_match, min_ratio = 90):
    # get a list of unique strings
    strings = df[column].unique()
    
    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

    # Only get matches with a ratio > 90
    close_matches = [matches[0] for matches in matches if matches[1] >= min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # Replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match
    
    # Let us know when the function is done
    print("All done!")


In [45]:
# Launch the function for every major variation
replace_matches_in_column(df=df, column='country', string_to_match="South Africa")
replace_matches_in_column(df=df, column='country', string_to_match="United Kingdom")
replace_matches_in_column(df=df, column='country', string_to_match="United States")
replace_matches_in_column(df=df, column='country', string_to_match="s.a.")
replace_matches_in_column(df=df, column='country', string_to_match="sa")
replace_matches_in_column(df=df, column='country', string_to_match="britain")
replace_matches_in_column(df=df, column='country', string_to_match="u.k.")
replace_matches_in_column(df=df, column='country', string_to_match="uk")
replace_matches_in_column(df=df, column='country', string_to_match="america")
replace_matches_in_column(df=df, column='country', string_to_match="england")
replace_matches_in_column(df=df, column='country', string_to_match="s. africasouth africa")
replace_matches_in_column(df=df, column='country', string_to_match="united states of america")

All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!
All done!


In [46]:
# Check how many unique values remains
unique_countries = df['country'].unique()
print(f"There are {len(unique_countries)} in the column")
unique_countries

There are 15 in the column


array(['United States', 'britain', 'United Kingdom', 'u.k.', 'sa',
       'america', 'Other', 's.a.', 'england', 'uk', '',
       'united states of america', 's. africasouth africa', '/', '.'],
      dtype=object)

In [47]:
# Get rid of the last unwanted variations manually replacing them
df.replace('uk', 'United Kingdom', inplace=True)
df.replace('u.k.', 'United Kingdom', inplace=True)
df.replace('britain', 'United Kingdom', inplace=True)
df.replace('england', 'United Kingdom', inplace=True)
df.replace('america', 'United States', inplace=True)
df.replace('united states of america', 'United States', inplace=True)
df.replace('sa', 'South Africa', inplace=True)
df.replace('s.a.', 'South Africa', inplace=True)
df.replace('s. africasouth africa', 'South Africa', inplace=True)
df.replace('/', 'Other', inplace=True)
df.replace('.', 'Other', inplace=True)
df.replace('', 'Other', inplace=True)

# Check how many unique values remains
unique_countries = df['country'].unique()
print(f"There are {len(unique_countries)} in the column")
unique_countries

There are 4 in the column


array(['United States', 'United Kingdom', 'South Africa', 'Other'],
      dtype=object)

3. Create a new column called `days_ago` in the DataFrame that is a copy of the 'date_measured' column but instead it is a number that shows how many days ago it was measured from the current date. Note that the current date can be obtained using `datetime.date.today()`.

In [48]:
from datetime import datetime

# Parse the date in a new column
df['date_parsed'] = pd.to_datetime(df['date_measured'], format='%d-%m-%Y')
# Create a column with the today date
df['today'] = str(date.today().strftime('%d-%m-%Y'))
# Parse the today date
df['today'] = pd.to_datetime(df['today'], format='%d-%m-%Y')
# Create a new column calculating the number of days between the current date and the date of the measuring
df['days_ago'] = (df['today'] - df['date_parsed']).dt.days